## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page in above app. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

In [ ]:
%pip install okareo 

In [1]:
# perform a test run using a scenario set
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, SeedData, ScenarioType
from okareo.model_under_test import CustomModel
from types import SimpleNamespace

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

class RetrievalModel(CustomModel):
    # Callable to be applied to each scenario in the scenario set
    def invoke(self, input: str):
        # call your model being tested here using <input> from the scenario set

        # mock code returning a random label
        labels = ["returns", "complaints", "pricing"]
        import random
        actual = random.choice(labels)

        return actual, {"labels": actual, "confidence": 0.8 }  # return a tuple of (actual, overall model response context)

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="intent_classifier", model=RetrievalModel(name="custom classification"))

# generate example scenario based on seed data and return results in one call
scenario_set_create = ScenarioSetCreate(name="My Test Scenario Set ",
                                        generation_type=ScenarioType.SEED,
                                        number_examples=1,
                                        seed_data=[SeedData(input_="I want to send this product back", result="returns"),
                                                   SeedData(input_="my product is not working", result="complaints"),
                                                   SeedData(input_="how much is this product?", result="pricing")])

source_scenario = okareo.create_scenario_set(scenario_set_create)
scenario_id = source_scenario.scenario_id

# use the scenario id to run the test
test_run_item = model_under_test.run_test(scenario=scenario_id, name="Intent Classifier Run 3", calculate_metrics=True)

# display model level metrics for the test run
print(test_run_item.model_metrics.to_dict())
print(test_run_item.app_link)

{'weighted_average': {'precision': 0.5, 'recall': 0.6666666666666666, 'f1': 0.5555555555555555, 'accuracy': 0.6666666666666666}, 'scores_by_label': {'pricing': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'complaints': {'precision': 0.5, 'recall': 1.0, 'f1': 0.6666666666666666}, 'returns': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}}
https://app.okareo.com/project/21b8a05b-f5b6-4578-a36a-fc264036d9d3/eval/d18fe542-2fb4-42c7-91a1-83cfeb83578d


In [2]:
# Retrieve metrics from an earlier test run
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="intent_classifier")
#test run id from the previous cell output
test_run_item = model_under_test.get_test_run(test_run_id=test_run_item.id)

# display model level metrics for the test run
print(test_run_item.model_metrics.to_dict())
print(test_run_item.app_link)

{'weighted_average': {'precision': 0.5, 'recall': 0.6666666666666666, 'f1': 0.5555555555555555, 'accuracy': 0.6666666666666666}, 'scores_by_label': {'pricing': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'complaints': {'precision': 0.5, 'recall': 1.0, 'f1': 0.6666666666666666}, 'returns': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}}
https://app.okareo.com/project/21b8a05b-f5b6-4578-a36a-fc264036d9d3/eval/d18fe542-2fb4-42c7-91a1-83cfeb83578d
